<a href="https://colab.research.google.com/github/Dinhthixuanbinh/Note/blob/main/Detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [833 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [977 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,231 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [865 kB]
Get:13 http://archive.ubunt

In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44091 sha256=a838cde44ee9afac96d22fedf206331f74ff430fec6e2e1de10321ef218b742c
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.7.0 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatible.
Cloning into 'detectron2'...
remote: Enumerating objects: 15180, done.
remote: Counting ob

In [3]:
!unzip /content/faster_rcnn_R_101_FPN_3x-20230803T015241Z-001.zip

unzip:  cannot find or open /content/faster_rcnn_R_101_FPN_3x-20230803T015241Z-001.zip, /content/faster_rcnn_R_101_FPN_3x-20230803T015241Z-001.zip.zip or /content/faster_rcnn_R_101_FPN_3x-20230803T015241Z-001.zip.ZIP.


In [5]:
!git clone https://github.com/CaseDrive/publaynet-models.git

Cloning into 'publaynet-models'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 339 (delta 85), reused 339 (delta 85), pack-reused 0
Receiving objects: 100% (339/339), 7.93 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch, detectron2
from detectron2 import model_zoo
import cv2
import numpy
from PIL import Image as image_main
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import os, shutil
import os
import pytesseract
from PIL import Image

!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [ ]:
os.makedirs('result/figure', exist_ok=True)
os.makedirs('result/text', exist_ok=True)


In [ ]:
image_path = '/content/test.png'
model_path = '/content/drive/MyDrive/data/Detectron2/faster_rcnn_R_101_FPN_3x/model_final.pth'
model_zoo_config_name = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'
prediction_score_threshold = 0.7
class_labels = ['text', 'title', 'list', 'table', 'figure']
folder = '/content/result'

In [16]:

# Detectron config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_zoo_config_name))
cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = prediction_score_threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

# Detectron predictor
predictor = DefaultPredictor(cfg)

# Open image, and run predictor
image_pil = image_main.open(image_path)
image_cv = cv2.cvtColor(numpy.array(image_pil), cv2.COLOR_RGB2BGR)
outputs = predictor(image_cv)

# Debug outputs
instances = outputs["instances"].to("cpu")
pred_boxes = instances.pred_boxes
scores = instances.scores
pred_classes = instances.pred_classes

image = cv2.imread(image_path)
image_copy = image.copy()
os.makedirs('result', exist_ok=True)


for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))


In [19]:


for i in range(0, len(pred_boxes)):
    box = pred_boxes[i].tensor.numpy()[0]
    score = round(float(scores[i].numpy()), 4)
    label_key = int(pred_classes[i].numpy())
    label = class_labels[label_key]
    x = int(box[0])
    y = int(box[1])
    w = int(box[2] - box[0])
    h = int(box[3] - box[1])

    x_1 = int(box[0])
    y_1 = int(box[1])
    x_2 = int(box[2])
    y_2 = int(box[3])
    extracted_region = image[y_1:y_2, x_1:x_2]
    if label == 'figure' or label == 'table':
        save_folder = 'result/figure'
        image_copy[y_1:y_2, x_1:x_2] = (255, 255, 255)
        cv2.imwrite('/content/result/text/modified_image.png', image_copy)


    cv2.imwrite(f'{save_folder}/{label}_{i}.png', extracted_region)

    print('Detected object of label=' + str(label) + ' with score=' + str(score) + ' and in box={x=' + str(x) + ', y=' + str(y) + ', w=' + str(w) + ', h=' + str(h) + '}')


Detected object of label=text with score=0.9996 and in box={x=22, y=375, w=517, h=90}
Detected object of label=text with score=0.9971 and in box={x=25, y=300, w=521, h=67}
Detected object of label=text with score=0.994 and in box={x=25, y=235, w=521, h=57}
Detected object of label=table with score=0.9022 and in box={x=28, y=82, w=499, h=124}
Detected object of label=title with score=0.8438 and in box={x=29, y=28, w=93, h=11}
Detected object of label=text with score=0.8398 and in box={x=43, y=206, w=493, h=23}
Detected object of label=text with score=0.8254 and in box={x=28, y=46, w=516, h=30}
Detected object of label=figure with score=0.811 and in box={x=19, y=88, w=529, h=112}


In [21]:


def extract_text_from_image(image_file: str) -> str:
    # Open the image file using PIL (Python Imaging Library)
    image = Image.open(image_file)
    # Use pytesseract to extract text from the image
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text

def save_text_to_txt(text: str, output_file: str, append=False):
    mode = 'a' if append else 'w'
    with open(output_file, mode, encoding='utf-8') as file:
        file.write(text)

def process_images_and_save_to_txt(image_dir: str, output_file: str):
    image_files = [file for file in os.listdir(image_dir) if file.endswith(".png")]
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        extracted_text = extract_text_from_image(image_path)
        save_text_to_txt(extracted_text, output_file, append=True)

def main():
    input_image_dir = "/content/result/text/"
    output_file_path = "/content/extracted_text.txt"

    process_images_and_save_to_txt(input_image_dir, output_file_path)
    print(f"Extracted text from all images saved to: {output_file_path}")



Extracted text from all images saved to: /content/extracted_text.txt


In [ ]:
if __name__ == '__main__':
    main()
